## Working with Json
1.  read Json (read_json)
2.  to Json (to_json)
3.  Json normalize

Most of the company use MongoDB database,Cassandara and many more noSql database where data are stored as key value pairs or nested
key value pairs. Now when we are reading data how will we convert it to dataframe and how will we analyze the data. Lets See...

In [1]:
data = '{"employee_name": "James", "email": "james@gmail.com", "job_profile": [{"title1":"Team Lead", "title2":"Sr. Developer"}]}'

In [2]:
type(data)

str

In [3]:
import pandas as pd
pd.read_json(data)      # in output we can see main key becomes column name and values store in it

,employee_name,email,job_profile
0,James,james@gmail.com,"{'title1': 'Team Lead', 'title2': 'Sr. Develop..."


In [4]:
pd.read_json(data,orient="Records")   # we saw above by default orientaion is records

,employee_name,email,job_profile
0,James,james@gmail.com,"{'title1': 'Team Lead', 'title2': 'Sr. Develop..."


In [5]:
pd.read_json(data,orient="index")

,0
employee_name,James
email,james@gmail.com
job_profile,"[{'title1': 'Team Lead', 'title2': 'Sr. Develo..."


In [8]:
df = pd.DataFrame([['a', 'b'], ['c', 'd']],
                  index=['row 1', 'row 2'],
                  columns=['col 1', 'col 2'])
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [10]:
df.to_json()

'{"col 1":{"row 1":"a","row 2":"c"},"col 2":{"row 1":"b","row 2":"d"}}'

In [12]:
df.to_json(orient='index')   #by default orient is index we infer from above

'{"row 1":{"col 1":"a","col 2":"b"},"row 2":{"col 1":"c","col 2":"d"}}'

In [14]:
df.to_json(orient='columns')

'{"col 1":{"row 1":"a","row 2":"c"},"col 2":{"row 1":"b","row 2":"d"}}'

In [15]:
df.to_json(orient='records')

'[{"col 1":"a","col 2":"b"},{"col 1":"c","col 2":"d"}]'

In [16]:
df.to_json(orient='split')

'{"columns":["col 1","col 2"],"index":["row 1","row 2"],"data":[["a","b"],["c","d"]]}'

In [18]:
df.to_json(orient='table')  # all the orientation basically is used in how you want to store your data

'{"schema":{"fields":[{"name":"index","type":"string"},{"name":"col 1","type":"string"},{"name":"col 2","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":"row 1","col 1":"a","col 2":"b"},{"index":"row 2","col 1":"c","col 2":"d"}]}'

In [19]:
data = [{"employee_name": "James", "email": "james@gmail.com", "job_profile": {"title1":"Team Lead", "title2":"Sr. Developer"}}]

In [20]:
type(data)

list

In [21]:
pd.json_normalize(data)

,employee_name,email,job_profile.title1,job_profile.title2
0,James,james@gmail.com,Team Lead,Sr. Developer


In [22]:
data = [
    {
        "id": 1,
        "name": "Cole Volk",
        "fitness": {"height": 130, "weight": 60},
    },
    {"name": "Mark Reg", "fitness": {"height": 130, "weight": 60}},
    {
        "id": 2,
        "name": "Faye Raker",
        "fitness": {"height": 130, "weight": 60},
    },
]

In [23]:
pd.json_normalize(data)

,id,name,fitness.height,fitness.weight
0,1.0,Cole Volk,130,60
1,NaN,Mark Reg,130,60
2,2.0,Faye Raker,130,60


In [24]:
pd.json_normalize(data,max_level= 0)  # if max level is zero it will only take the first key pair as column name

,id,name,fitness
0,1.0,Cole Volk,"{'height': 130, 'weight': 60}"
1,NaN,Mark Reg,"{'height': 130, 'weight': 60}"
2,2.0,Faye Raker,"{'height': 130, 'weight': 60}"


In [28]:
pd.json_normalize(data,max_level= 1)  # if max level is one it will go one level down of the nested dict
                                    # note: max level dosent work if you have a list inside key value pair
                                    

,id,name,fitness.height,fitness.weight
0,1.0,Cole Volk,130,60
1,NaN,Mark Reg,130,60
2,2.0,Faye Raker,130,60


In [29]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]

In [30]:
pd.json_normalize(data)

,state,shortname,counties,info.governor
0,Florida,FL,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott
1,Ohio,OH,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich


In [33]:
#But i want to read data that is under column "counties" too then...
# we will use parameter 'record_path'
pd.json_normalize(data,"counties")

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [38]:
# But we want all data ? 
# so, there is aparameter called 'meta'
pd.json_normalize(data,"counties" ,["state","shortname","info"])

,name,population,state,shortname,info
0,Dade,12345,Florida,FL,{'governor': 'Rick Scott'}
1,Broward,40000,Florida,FL,{'governor': 'Rick Scott'}
2,Palm Beach,60000,Florida,FL,{'governor': 'Rick Scott'}
3,Summit,1234,Ohio,OH,{'governor': 'John Kasich'}
4,Cuyahoga,1337,Ohio,OH,{'governor': 'John Kasich'}


In [47]:
pd.json_normalize(data,"counties" ,["state","shortname",["info","governor"]])


,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich
